In [2]:
# Import Dependencies
import numpy as np
import pandas as pd
import pyspark as spark
import sklearn as skl
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Create pandas dataframe from the csv file
df = pd.read_csv('Pseo_All.csv')

In [6]:
df.head()

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,status_y1_earnings,status_y5_earnings,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate
0,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
1,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
2,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
3,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
4,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0


In [7]:
df.dtypes

agg_level_pseo                   int64
inst_level                      object
institution                      int64
degree_level                     int64
cip_level                       object
cipcode                          int64
grad_cohort                      int64
grad_cohort_years                int64
geo_level                       object
geography                        int64
ind_level                       object
industry                         int64
y1_p50_earnings                  int64
y5_p50_earnings                  int64
y1_ipeds_count                   int64
y5_ipeds_count                   int64
status_y1_earnings               int64
status_y5_earnings               int64
y1_grads_emp                   float64
y1_grads_emp_instate           float64
y5_grads_emp                   float64
y5_grads_emp_instate           float64
status_y1_grads_emp            float64
status_y1_grads_emp_instate    float64
status_y5_grads_emp            float64
status_y5_grads_emp_insta

In [8]:
df.groupby('inst_level').inst_level.count()

inst_level
I    223160
Name: inst_level, dtype: int64

In [9]:
df.groupby('geo_level').geo_level.count()

geo_level
N    223160
Name: geo_level, dtype: int64

In [10]:
df.groupby('cip_level').cip_level.count()

cip_level
2    97984
4    26872
2    58122
4    17994
A    22188
Name: cip_level, dtype: int64

In [11]:
df.groupby('ind_level').ind_level.count()

ind_level
A    223160
Name: ind_level, dtype: int64

In [12]:
# Since inst_level, geo_level, and ind level are all the same value they are dropped from the dataframe
dropped_df= df.drop(['inst_level', 'geo_level', 'ind_level'], axis = 1)
dropped_df.head()

,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry,y1_p50_earnings,...,status_y1_earnings,status_y5_earnings,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate
0,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
1,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
2,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
3,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
4,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0


In [13]:
#Replacing all "A" values with the number 1 so that they are all the same data type
dropped_df.loc[dropped_df["cip_level"] == "A", "cip_level"] = 1

# Changing data type
dropped_df['cip_level']= dropped_df['cip_level'].astype(int)
dropped_df.dtypes

agg_level_pseo                   int64
institution                      int64
degree_level                     int64
cip_level                        int64
cipcode                          int64
grad_cohort                      int64
grad_cohort_years                int64
geography                        int64
industry                         int64
y1_p50_earnings                  int64
y5_p50_earnings                  int64
y1_ipeds_count                   int64
y5_ipeds_count                   int64
status_y1_earnings               int64
status_y5_earnings               int64
y1_grads_emp                   float64
y1_grads_emp_instate           float64
y5_grads_emp                   float64
y5_grads_emp_instate           float64
status_y1_grads_emp            float64
status_y1_grads_emp_instate    float64
status_y5_grads_emp            float64
status_y5_grads_emp_instate    float64
dtype: object

In [14]:
# One hot encoding is used on cip_level column

# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(dropped_df.cip_level.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['cip_level'])
encode_df.head()

,cip_level_1,cip_level_2,cip_level_4
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [15]:
# Merge the two DataFrames together and drop the cip_level column
dropped_df.merge(encode_df,left_index=True,right_index=True).drop("cip_level",1)

,agg_level_pseo,institution,degree_level,cipcode,grad_cohort,grad_cohort_years,geography,industry,y1_p50_earnings,y5_p50_earnings,...,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate,cip_level_1,cip_level_2,cip_level_4
0,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223155,48,3730300,3,52,2001,5,0,0,26840,35645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
223156,48,3789400,3,24,2001,5,0,0,26167,39475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
223157,48,3789400,3,24,2001,5,0,0,26167,39475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
223158,48,3956300,3,13,2001,5,0,0,29553,31238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0


In [16]:
# Define features dataset
X= dropped_df.drop(['y1_p50_earnings','y5_p50_earnings', 'y1_ipeds_count', 'status_y1_earnings','status_y5_earnings','y1_grads_emp','y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 'status_y5_grads_emp','y5_ipeds_count','status_y5_grads_emp_instate'],axis= 1)
X.head()


,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry
0,38,105100,5,1,0,0,3,0,0
1,38,105100,5,1,0,0,3,0,0
2,38,105100,5,1,0,0,3,0,0
3,38,105100,5,1,0,0,3,0,0
4,38,105100,5,1,0,0,3,0,0


In [17]:
# Define target dataset
y = dropped_df["y1_p50_earnings"].ravel()
y[:5]

array([37267, 37267, 37267, 37267, 37267])

In [18]:
# Splitting into Train and Test sets for Logistic regression.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=len(X_train_scaled[0])))

# Add a hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                800       
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 4,081
Trainable params: 4,081
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn_model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["mean_absolute_error"])

In [21]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
5231/5231 [==============================] - 11s 2ms/step - loss: 13835.5693 - mean_absolute_error: 13835.5693
Epoch 2/25
5231/5231 [==============================] - 9s 2ms/step - loss: 8314.9551 - mean_absolute_error: 8314.9551
Epoch 3/25
5231/5231 [==============================] - 8s 2ms/step - loss: 8257.2549 - mean_absolute_error: 8257.2549
Epoch 4/25
5231/5231 [==============================] - 8s 1ms/step - loss: 8227.2734 - mean_absolute_error: 8227.2734
Epoch 5/25
5231/5231 [==============================] - 8s 1ms/step - loss: 8203.8740 - mean_absolute_error: 8203.8740
Epoch 6/25
5231/5231 [==============================] - 9s 2ms/step - loss: 8189.9497 - mean_absolute_error: 8189.9497
Epoch 7/25
5231/5231 [==============================] - 8s 2ms/step - loss: 8176.8301 - mean_absolute_error: 8176.8301
Epoch 8/25
5231/5231 [==============================] - 8s 2ms/step - loss: 8165.9878 - mean_absolute_error: 8165.9878
Epoch 9/25
5231/5231 [=======================

In [22]:
# Define features dataset
X= dropped_df.drop(['y1_p50_earnings','y5_p50_earnings', 'y1_ipeds_count', 'status_y1_earnings','status_y5_earnings','y1_grads_emp','y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 'status_y5_grads_emp','y5_ipeds_count','status_y5_grads_emp_instate'],axis= 1)
X.head()


,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry
0,38,105100,5,1,0,0,3,0,0
1,38,105100,5,1,0,0,3,0,0
2,38,105100,5,1,0,0,3,0,0
3,38,105100,5,1,0,0,3,0,0
4,38,105100,5,1,0,0,3,0,0


In [23]:
# Define target dataset
y = dropped_df["y5_p50_earnings"].ravel()
y[:5]

array([51849, 51849, 51849, 51849, 51849])

In [24]:
# Splitting into Train and Test sets for Logistic regression.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=len(X_train_scaled[0])))

# Add a hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                800       
                                                                 
 dense_7 (Dense)             (None, 40)                3240      
                                                                 
 dense_8 (Dense)             (None, 1)                 41        
                                                                 
Total params: 4,081
Trainable params: 4,081
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn_model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["mean_absolute_error"])

In [27]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
5231/5231 [==============================] - 9s 2ms/step - loss: 19749.9355 - mean_absolute_error: 19749.9355
Epoch 2/25
5231/5231 [==============================] - 8s 1ms/step - loss: 9742.5449 - mean_absolute_error: 9742.5449
Epoch 3/25
5231/5231 [==============================] - 8s 1ms/step - loss: 9628.8193 - mean_absolute_error: 9628.8193
Epoch 4/25
5231/5231 [==============================] - 8s 2ms/step - loss: 9590.3984 - mean_absolute_error: 9590.3984
Epoch 5/25
5231/5231 [==============================] - 9s 2ms/step - loss: 9571.7178 - mean_absolute_error: 9571.7178
Epoch 6/25
5231/5231 [==============================] - 8s 2ms/step - loss: 9561.7197 - mean_absolute_error: 9561.7197
Epoch 7/25
5231/5231 [==============================] - 8s 2ms/step - loss: 9554.4590 - mean_absolute_error: 9554.4590
Epoch 8/25
5231/5231 [==============================] - 9s 2ms/step - loss: 9546.9346 - mean_absolute_error: 9546.9346
Epoch 9/25
5231/5231 [========================

In [42]:
# Applying the condition
dropped_df["y1_p50_earnings"] = np.where(df["y1_p50_earnings"] < 35000, 0, 1)
dropped_df.groupby('y1_p50_earnings').y1_p50_earnings.count()

y1_p50_earnings
0    114374
1    108786
Name: y1_p50_earnings, dtype: int64

In [43]:
# Prepare for test on year 5 earnings
dropped_df["y5_p50_earnings"] = np.where(df["y5_p50_earnings"] < 50000, 0, 1)
dropped_df.groupby('y5_p50_earnings').y1_p50_earnings.count()

y5_p50_earnings
0    116852
1    106308
Name: y1_p50_earnings, dtype: int64

In [30]:
# Define features dataset
X= dropped_df.drop(['y1_p50_earnings','y5_p50_earnings', 'y1_ipeds_count', 'status_y1_earnings','status_y5_earnings','y1_grads_emp','y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 'status_y5_grads_emp','y5_ipeds_count','status_y5_grads_emp_instate'],axis= 1)
X.head()


,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry
0,38,105100,5,1,0,0,3,0,0
1,38,105100,5,1,0,0,3,0,0
2,38,105100,5,1,0,0,3,0,0
3,38,105100,5,1,0,0,3,0,0
4,38,105100,5,1,0,0,3,0,0


In [31]:
# Define target dataset
y = dropped_df["y1_p50_earnings"].ravel()
y[:5]

array([1, 1, 1, 1, 1])

In [32]:
# Splitting into Train and Test sets for Logistic regression.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=len(X_train_scaled[0])))

# Add a hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                800       
                                                                 
 dense_10 (Dense)            (None, 40)                3240      
                                                                 
 dense_11 (Dense)            (None, 1)                 41        
                                                                 
Total params: 4,081
Trainable params: 4,081
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [35]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5965 - accuracy: 0.6620
Epoch 2/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5769 - accuracy: 0.6841
Epoch 3/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5623 - accuracy: 0.6932
Epoch 4/25
5231/5231 [==============================] - 8s 2ms/step - loss: 0.5498 - accuracy: 0.7070
Epoch 5/25
5231/5231 [==============================] - 8s 2ms/step - loss: 0.5380 - accuracy: 0.7201
Epoch 6/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5262 - accuracy: 0.7314
Epoch 7/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5166 - accuracy: 0.7406
Epoch 8/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5077 - accuracy: 0.7476
Epoch 9/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5006 - accuracy: 0.7533
Epoch 10/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.494

In [36]:
# Define features dataset
X= dropped_df.drop(['y1_p50_earnings','y5_p50_earnings', 'y1_ipeds_count', 'status_y1_earnings','status_y5_earnings','y1_grads_emp','y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 'status_y5_grads_emp','y5_ipeds_count','status_y5_grads_emp_instate'],axis= 1)
X.head()


,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry
0,38,105100,5,1,0,0,3,0,0
1,38,105100,5,1,0,0,3,0,0
2,38,105100,5,1,0,0,3,0,0
3,38,105100,5,1,0,0,3,0,0
4,38,105100,5,1,0,0,3,0,0


In [37]:
# Define target dataset
y = dropped_df["y5_p50_earnings"].ravel()
y[:5]

array([1, 1, 1, 1, 1])

In [38]:
# Splitting into Train and Test sets for Logistic regression.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=len(X_train_scaled[0])))

# Add a hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                800       
                                                                 
 dense_13 (Dense)            (None, 40)                3240      
                                                                 
 dense_14 (Dense)            (None, 1)                 41        
                                                                 
Total params: 4,081
Trainable params: 4,081
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [41]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.6115 - accuracy: 0.6462
Epoch 2/25
5231/5231 [==============================] - 8s 2ms/step - loss: 0.5937 - accuracy: 0.6675
Epoch 3/25
5231/5231 [==============================] - 8s 2ms/step - loss: 0.5774 - accuracy: 0.6799
Epoch 4/25
5231/5231 [==============================] - 10s 2ms/step - loss: 0.5637 - accuracy: 0.6957
Epoch 5/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5534 - accuracy: 0.7083
Epoch 6/25
5231/5231 [==============================] - 9s 2ms/step - loss: 0.5428 - accuracy: 0.7213
Epoch 7/25
5231/5231 [==============================] - 10s 2ms/step - loss: 0.5323 - accuracy: 0.7307
Epoch 8/25
5231/5231 [==============================] - 11s 2ms/step - loss: 0.5233 - accuracy: 0.7372
Epoch 9/25
5231/5231 [==============================] - 10s 2ms/step - loss: 0.5147 - accuracy: 0.7439
Epoch 10/25
5231/5231 [==============================] - 10s 2ms/step - loss: 